In [1]:
# 프로젝트 폴더에 'data'폴더를 저장
# data밑에 test_set, training_set 폴더 저장 
# 그 밑에 각각 cats, dogs 폴더 저장 후 그 안에 사진파일들이 있어야함
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


base_dir = './cat/images/'

In [2]:
# 데이터셋에서 이미지 로드 및 확인
def check_images(directory):
    total_files = sum([len(files) for r, d, files in os.walk(directory)])
    with tqdm(total=total_files, desc="Checking images") as pbar:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(('jpg', 'jpeg', 'png')):
                    img_path = os.path.join(root, file)
                    try:
                        img = load_img(img_path, target_size=(224, 224))  # 이미지 로드 및 크기 조정
                        img_array = img_to_array(img)  # 배열로 변환
                        img_array = img_array / 255.0  # 정규화
                        if img_array.shape != (224, 224, 3): # 이미지 배열의 형태를 검사
                            raise ValueError(f"Unexpected image shape: {img_array.shape}")
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")
                    pbar.update(1)

# 이미지 체크
check_images(base_dir)

Checking images: 100%|██████████| 120887/120887 [01:48<00:00, 1115.57it/s]


In [3]:
# 데이터 전처리 (이미지 개수가 충분하므로 증강까지는 안함)
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2 # 훈련과 검증용으로 쪼개기
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    interpolation='nearest'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    interpolation='nearest'
)

Found 96727 images belonging to 46 classes.
Found 24160 images belonging to 46 classes.


In [4]:
# InceptionResNetV2 모델을 로드하고, include_top=False로 설정하여 FC 레이어를 포함하지 않음
input_tensor = Input(shape=(224, 224, 3))
base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 베이스 모델의 일부 층을 동결
for layer in base_model.layers[:100]:  # 상위 100개 층을 동결
    layer.trainable = False

# 베이스 모델의 출력에 GlobalAveragePooling2D 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 새로운 출력 레이어 추가
num_classes = train_generator.num_classes # 클래스 수 자동 감지
output_tensor = Dense(num_classes, activation='softmax')(x)

# 모델 정의
model = Model(inputs=base_model.input, outputs=output_tensor)

# 모델 요약 출력
model.summary()

C:\Users\kimsu\AppData\Local\Temp\ipykernel_21356\3611231531.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,316,910 (8.84 MB)

 Trainable params: 1,920,366 (7.33 MB)

 Non-trainable params: 396,544 (1.51 MB)

In [5]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

In [6]:
# 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [7]:
epochs = 30
model.fit(
    train_generator,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/30


C:\Users\kimsu\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  35/1512 ━━━━━━━━━━━━━━━━━━━━ 7:28 304ms/step - accuracy: 0.3464 - loss: 2.8695

KeyboardInterrupt: 

In [6]:
model.save('cat_breeds.keras')